In [1]:
import os
import math
import numpy as np
import pandas as pd
import re
import pickle
import matplotlib.pyplot as plt

In [2]:
cur_path = os.getcwd()
file_path = cur_path + "\Documents\\technical-assignment-2020-0823"
train_data_path = file_path + '\\test_data'

valid_data_path = file_path + '\\test_data'

In [9]:
# read train data
test_data = pd.read_csv(valid_data_path, sep='\n', names=["review"], header=None, skip_blank_lines=False)

In [12]:
test_data.head

<bound method NDFrame.head of                                                    review
0                               진짜 가만히 있어도 눈물이 나옵니다ㅠㅠㅠㅠㅠㅠ
1       영화를 분석하며 보는 불쌍한 사람이 되지 마세요 아이 키우는 아빠로서 많은 눈물을 ...
2                   정말 다시보고싶은영화   적극추천합니다배우들연기 짱  기대이상입니다
3                                  대박 ㅠㅠ웃음에서감동까지 최고의영화입니다
4                        감동있지만 개인적으로 지적장애로 유발된 웃음은 싫어하는터라
...                                                   ...
399995                   같은여자인데 엘리제한테반햇어요 노래도좋고 힐링되는영화였어요
399996           믿음을 부서트리기 위한 악마의 장치 브로큰 써클 욥기를 다시금 되새겨본다
399997                             나도 이렇게 멋진 사랑을 하고 싶다아아앙
399998                                         삐료 정말화가난다 
399999                      맨날 울고불고 꼭 누구 하나 죽고 뻔한 구닥다리 방화

[400000 rows x 1 columns]>

In [11]:
# replace all other string to blank except korean
test_data['review'] = test_data['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

#test_data['review'].replace('', '굳', inplace=True)
test_data = test_data.fillna('굳')
#test_data = test_data.dropna(how = 'any')


print('전처리 후 샘플의 개수 :',len(test_data))

전처리 후 샘플의 개수 : 400000


In [13]:
from konlpy.tag import Okt
okt = Okt()

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [14]:
# 토큰화 객체 계속 쓰기
X_test = []
for sentence in test_data['review']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)
print(len(X_test))

400000


In [15]:
with open('test_tokenized_data.p', 'wb') as file:
    pickle.dump(X_test, file)

In [2]:
# 토큰화 객체 불러오기
with open('tokenized_data1.p', 'rb') as file:
    X_train = pickle.load(file)
    
with open('data1_label.p', 'rb') as file:
    y_train = pickle.load(file)
    
with open('test_tokenized_data.p', 'rb') as file:
    X_test = pickle.load(file)
    
print(len(X_train))
print(len(y_train))
print(len(X_test))

84222
84222
400000


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 29531
등장 빈도가 2번 이하인 희귀 단어의 수: 17187
단어 집합에서 희귀 단어의 비율: 58.19985777657377
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.3858058875623716


In [4]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

단어 집합의 크기 : 12346


In [6]:
X_train = pad_sequences(X_train, maxlen = 30)
X_test = pad_sequences(X_test, maxlen = 30)

In [27]:
from tensorflow.keras.models import load_model

loaded_model = load_model('best_model.h5')
y_test = []

def sentiment_predict(new_sentence):
    score = float(loaded_model.predict(new_sentence)) # 예측
    y_test.append(math.ceil(score*10))

for sentence in X_test:
    sentiment_predict(np.array([sentence]))

In [28]:
cur_path = os.getcwd()
file_path = cur_path + "\Documents\\technical-assignment-2020-0823"
submission_file_path = file_path + '\\submission.csv'

submission = pd.DataFrame(y_test, columns=["prediction"])
submission.to_csv(submission_file_path)

In [89]:
loaded_model = load_model('best_model.h5')

def sentiment_predict(new_sentence):
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = 30) # 패딩
    print(type(pad_new))
    print(loaded_model.predict(pad_new))
    score = float(loaded_model.predict(pad_new)) # 예측
    print(score)
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

sentiment_predict('이 영화 핵노잼 ㅠㅠ')

<class 'numpy.ndarray'>
[[0.00227015]]
0.0022701455745846033
99.77% 확률로 부정 리뷰입니다.



In [18]:
file_path = cur_path + "\Documents\\technical-assignment-2020-0823"
submission_file = file_path + '\\submission0911.csv'

In [22]:
sub = pd.read_csv(submission_file, header=0, usecols=[1,2])

In [26]:
sub.head

<bound method NDFrame.head of             ID  prediction
0            0          10
1            1          10
2            2          10
3            3          10
4            4           8
...        ...         ...
399995  399995          10
399996  399996          10
399997  399997          10
399998  399998          10
399999  399999           8

[400000 rows x 2 columns]>

In [27]:
sub.to_csv(submission_file, index=False)